### Import Required Packages and Set Options
Use Matrix Factorization with attributes to suggest destinations

#### Import Base Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import random
import function_lib as flib

import numpy as np
#import numba as nb
import pandas as pd
# from scipy.sparse import csr_matrix

import matplotlib.pyplot as plt
import seaborn as sns

import rankfmlib as rfmlib

In [109]:
from rankfm.rankfm import RankFM
from rankfm.evaluation import hit_rate, reciprocal_rank, discounted_cumulative_gain, precision, recall, diversity

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', '{:.3f}'.format)

sns.set_style('whitegrid')
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
# plt.rcParams

#### Put the Main Package Library on the PYTHONPATH

In [5]:
# git_repo = git.Repo('.', search_parent_directories=True)
# git_root = git_repo.git.rev_parse('--show-toplevel')
# cython_path = os.path.join(git_root, 'rankfm')

# sys.path[0] = git_root
# sys.path[1] = cython_path
# sys.path[:2]

#### Re-Compile Cython Extension Module

In [6]:
# !cd $git_root && python setup.py build_ext --inplace

#### Dynamically Re-Load all Package Modules on Execution

#### Set File Path Constants

In [7]:
!pwd

/home/erlebach/src/2022/copa_recommenders/copa_recommender_data_rankfm/one_destination_per_member


In [8]:
data_path = "." # os.path.join(git_root, "data/instacart_2017_05_01")
# print("\n".join([git_root, data_path]))

### Prepare Copa Data
* Data for one year is included in each file.
* Each file has three columns: userID (memberID), itemID (destination), rating (always 1)
* The data was produced elsewhere (perhaps on my mac, perhaps not)

#### Load Departments Data
We will use 2016 for training and 2017 for testing

In [39]:
df_member = {}
df_attrib = {}
year_train = 2016
year_valid = 2017
df_member_train, df_attrib_train = rfmlib.read_data_attributes(year_train)
df_member_valid, df_attrib_valid = rfmlib.read_data_attributes(year_valid)

## First, analyze 2016, training data

In [44]:
# I should probably randomize
shuffle_index = np.arange(len(interactions))
print(shuffle_index)
np.random.shuffle(shuffle_index)
interactions = df_member_train.copy().iloc[shuffle_index].reset_index(drop=True)     
attributes = df_attrib_train.copy().iloc[shuffle_index].reset_index(drop=True)

[     0      1      2 ... 121803 121804 121805]


In [47]:
dct = rfmlib.sparsity(interactions)
print(dct)

{'sparsity': 0.9648460524660084, 'n_users': 23841, 'n_items': 76}


### Take a Random Subsample of Users

In [57]:
all_users = interactions.user_id.unique()
all_users1 = attributes.user_id.unique()
assert(len(all_users) == len(all_users1))

In [58]:
np.random.seed(1492)
nb_users = len(all_users)
keep_nb_users = 10000
# keep_nb_users = nb_users   # Keep all the users
# shuffle the users
s_users = np.random.choice(all_users, size=keep_nb_users, replace=False)

Keep only the trips of the users selected

In [100]:
s_interactions = interactions[interactions.user_id.isin(s_users)].copy()
s_attributes = attributes[interactions.user_id.isin(s_users)].copy()
s_interactions.shape, interactions.shape

((51146, 2), (121806, 2))

In [107]:
# Difference is zero indicating equality. I ensured that users were ordered the same way in both dataframes
(s_attributes.user_id.values - s_interactions.user_id.values).sum()

0

Number of destinations travelled by the users kept in our selection

In [66]:
s_items = s_interactions.product_id.unique()
print("nb unique destinations: ", len(s_items))

nb unique destinations:  76


In [108]:
RamkFM

NameError: name 'RamkFM' is not defined

Maximum number of destinations for a single user

In [71]:
s_interactions.groupby(['user_id', 'product_id']).size().max()

46

In [81]:
df_nb_trips = s_interactions.groupby(['user_id'])['product_id'].size().to_frame('nb_trips')
df_nb_trips[df_nb_trips['nb_trips'] > 40]

,nb_trips
user_id,
230003257,43
230161925,41
230340635,42
231120328,43
231204984,43
231429942,42
231449144,46
231981026,46


Users taking many trips during the year are of two types: either they go to 2-3 destinations or they go to many different destinations. What are the characteristics of these users?

In [90]:
s_interactions[s_interactions['user_id'] == 231449144].head()  # remove head() for more destinations

,user_id,product_id
8255,231449144,EZE
10057,231449144,COR
11880,231449144,SFO
13051,231449144,LIM
22825,231449144,BOG


In [125]:
model = RankFM(factors=20, loss='warp', max_samples=50, alpha=0.01, learning_rate=0.1, learning_schedule='invscaling')

In [127]:
%%time
# sample_weight_train = interactions_dct["sample_weight_train"]
# item_features = 
sample_weight_train = np.ones(s_interactions.shape[0])
model.fit(s_interactions, sample_weight=sample_weight_train, epochs=30, verbose=False)

CPU times: user 1.05 s, sys: 0 ns, total: 1.05 s
Wall time: 1.05 s


In [119]:
valid_users = s_users[0:]
valid_users

array([232182733, 230058605, 232633312, ..., 235675858, 230008857,
       230046388])

In [120]:
%%time
valid_recs = model.recommend(valid_users, n_items=4, filter_previous=False, cold_start='nan')

CPU times: user 103 ms, sys: 686 µs, total: 104 ms
Wall time: 103 ms


In [121]:
valid_recs

,0,1,2,3
232182733,GRU,EZE,SFO,MIA
230058605,VVI,SDQ,GUA,MIA
232633312,JFK,EZE,MGA,MIA
231248896,LIM,GUA,MEX,SDQ
231829196,SJO,GUA,MGA,MAR
...,...,...,...,...
230052725,GUA,SJO,LAS,TPA
231297767,SJO,POS,SDQ,BZE
235675858,GRU,MEX,BOG,MIA
230008857,PTY,MCO,LAS,SDQ


# END OF NEW CODE

#### Create a User/Item Interaction Data Set with all years

In [ ]:
df_g = {}
for year in years:
    df_g[year] = df[year].groupby(['user_id', 'product_id'])

interactions = pd.concat([df[year] for year in years], axis=0)

#### Check that there is only one entry per member_id/destination/year group  

In [ ]:
interactions.groupby(['user_id','product_id','year']).size().max()

In [ ]:
# orders_cols = ['order_id', 'user_id']
# order_products_cols = ['order_id', 'product_id']
# interaction_cols = ['user_id', 'product_id', 'order_id']

# interactions = pd.merge(orders_df[orders_cols], order_products_df[order_products_cols], on='order_id', how='inner')
# interactions = interactions[interaction_cols]

# interactions.info()
# interactions.head()

#### Evaluate User/Item Interaction Sparsity

In [ ]:
df[2016].shape, df[2017].shape

In [ ]:
dct = ranklib.sparsity(interactions)
print("full interaction data sparsity: {}".format(round(100 * dct['sparsity'], 2)))
print(f"n_users: {dct['n_users']}, n_items: {dct['n_items']}")

### Subsample the Data for Initial Testing

#### Take a Random Subsample of Users

In [ ]:
np.random.seed(1492)
nb_users = len(all_users)
# keep_nb_users = 10000
keep_nb_users = nb_users   # Keep all the users
# shuffle the users
s_users = np.random.choice(all_users, size=keep_nb_users, replace=False)

In [ ]:
len(s_users), len(set(s_users))

#### Get All Interactions for Those Users
Only keep the users present in both years (NOT IMPLEMENTED)

In [ ]:
s_interactions = interactions[interactions.user_id.isin(s_users)].copy()
s_interactions.shape, interactions.shape

In [ ]:
s_interacts.

In [ ]:
# Number of destinations travelled by users kept
s_items = s_interactions.product_id.unique()
len(s_items)

In [ ]:
len(list(set(interactions.index)))

In [ ]:
interactions.groupby(['user_id', 'product_id']).size()

I expected the maximum number of times a given member flew to a particular destination to be 1. 
Is it the case that when a member travels to a destination twice, it is in different years? Yes it is. 

In [ ]:
# Maximum number of times a given member flew to a particular destination is 2
interactions.groupby(['user_id', 'product_id', 'year']).size().max()

#### Re-Evaluate Cardinality/Sparsity on the Sample

In [ ]:
n_s_users = len(s_users)
n_s_items = len(s_items)

print("sample users:", n_s_users)
print("sample items:", n_s_items)
print("sample interactions:", s_interactions.shape)

In [ ]:
s_sparsity = 1 - (s_interactions[['user_id', 'product_id']].drop_duplicates().shape[0] / (n_s_users * n_s_items))
print("sample interaction data sparsity: {}".format(round(100 * s_sparsity, 2)))

### Split the Data into Training/Validation Sets

#### Randomly Shuffle the Overall Interaction Data

def shuffle_interaction_data(s_interactions):
    shuffle_index = np.arange(len(s_interactions))
    np.random.shuffle(shuffle_index)
    s_interactions = s_interactions.copy().iloc[shuffle_index]
    return s_interactions

#### Define training and validation data

In [ ]:
shuffled_interactions, shuffle_index = rfmlib.shuffle_interaction_data(s_interactions)
interactions_dct = rfmlib.train_validation(shuffled_interactions, 2016, 2017, shuffle_index)

In [38]:
interactions_dct.keys()

NameError: name 'interactions_dct' is not defined

In [ ]:
rfmlib.print_stats(interactions_dct)

### Test Out Core Package Functionality

#### Initialize the Model with Chosen Hyperparameters

In [ ]:
%%time 
# max_samples=500 creates problem for 'warp', but not for 'bpr'. Or vce-versa. What is the difference? And Why?
# max_samples: nb negative samples
model = RankFM(factors=50, loss='warp', max_samples=50, alpha=0.01, learning_rate=0.1, learning_schedule='invscaling')

#### Fit the Model on the Training Data and Profile Computational Performance

In [ ]:
%%time
sample_weight_train = interactions_dct["sample_weight_train"]
model.fit(interactions_train, sample_weight=sample_weight_train, epochs=30, verbose=False)

#### Generate Model Scores

In [ ]:
%%time
scores = model.predict(interactions_valid, cold_start='nan') # 'nan' or 'drop'

In [ ]:
scores.shape, scores[2], len(interactions_train), len(interactions_valid)

In [ ]:
scores[0:100], len(scores), len(interactions_valid)

#### Generate TopN Recommendations

In [ ]:
train_users = pd.Series(interactions_train.user_id.unique())
valid_users = pd.Series(interactions_valid.user_id.unique())
both_users = set(train_users) & set(valid_users)
cold_start_users = set(valid_users) - set(train_users)

len(train_users), len(valid_users), len(both_users), len(cold_start_users), 

In [ ]:
%%time
valid_recs = model.recommend(valid_users, n_items=10, filter_previous=False, cold_start='nan')

In [ ]:
%%time
valid_recs_filter_prev = model.recommend(valid_users, n_items=10, filter_previous=True, cold_start='nan')

In [ ]:
print(valid_recs.shape)
valid_recs.head()

In [ ]:
print(valid_recs_filter_prev.shape)
valid_recs_filter_prev.head()

### Evaluate Model Performance on the Validation Data

#### Specify Number of Recommended Items

In [ ]:
k = 3
topN = k

#### Generate Pure-Popularity Baselines

In [ ]:
most_popular = interactions_train.groupby('product_id')['user_id'].count().sort_values(ascending=False)[:k]
most_popular

In [ ]:
%%time 
# Create dictionary: member => list of destinations 
# Works even when there are there are multiple entries for the same (user_id, product_id)
# 1/3 sec
test_user_items = interactions_valid.groupby('user_id')['product_id'].apply(set)   # .to_dict()

In [ ]:
%%time 
# Only keep users in the training set
# 5.29 ms
test_user_items = test_user_items.loc[list(both_users)].to_dict()

In [ ]:
%%time  
# Only keep users contained in the training set
# Create dictionary: member => list of items
# Take 54 sec
#test_user_items = {key: val for key, val in test_user_items.items() if key in set(train_users)}

In [ ]:
%%time  
# Only keep users contained in the training set
# Create dictionary: member => list of items
# Take 54 sec because of python loops
# test_user_items1 = {key: val for key, val in test_user_items.items() if key in set(train_users)}

In [ ]:
%%time
base_hrt = np.mean([int(len(set(most_popular.index) & set(val)) > 0)                       for key, val in test_user_items.items()])
base_pre = np.mean([len(set(most_popular.index) & set(val)) / len(set(most_popular.index)) for key, val in test_user_items.items()])
base_rec = np.mean([len(set(most_popular.index) & set(val)) / len(set(val))                for key, val in test_user_items.items()])

In [ ]:
print("number of test users: {}".format(len(test_user_items)))
print("baseline hit rate: {:.3f}".format(base_hrt))
print("baseline precision: {:.3f}".format(base_pre))
print("baseline recall: {:.3f}".format(base_rec))

#### Generate Model Performance Validation Metrics

In [ ]:
%%time
model_hrt = hit_rate(model, interactions_valid, k=k)
model_rnk = reciprocal_rank(model, interactions_valid, k=k)
model_pre = precision(model, interactions_valid, k=k)
model_rec = recall(model, interactions_valid, k=k)

In [ ]:
print("model hit rate: {}".format(round(model_hrt, 3)))
print("model reciprocal rank: {}".format(round(model_rnk, 3)))
print("model precision: {}".format(round(model_pre, 3)))
print("model recall: {}".format(round(model_rec, 3)))

In [ ]:
# interactions_valid.groupby('user_id')['product_id'].apply(set)   # .to_dict()
interactions_train_Dlist = interactions_train.groupby('user_id')['product_id'].apply(set)

In [ ]:
# import function_lib as flib

In [ ]:
model_hrt = flib.hit_rate(model, interactions_valid, k=5, filter_previous=False, max_kept=1, train_interactions=interactions_train_Dlist)
# same function as in rankfm

In [ ]:
%%time
# average number of users for which at least one recommendation is correct
# 77% hit rate! (at least one hit correct)
# 26.6% hit rate (at least two hits correct)
# 5.6% (three hits correct)
# 
model_hrt = flib.hit_rate(model, interactions_valid, k=k, filter_previous=False)  # same function as in rankfm
print("model_hrt(filter_previous=False): ", model_hrt)
model_hrt = flib.hit_rate(model, interactions_valid, k=k, filter_previous=True)  # same function as in rankfm
print("model_hrt(filter_previous=True): ", model_hrt)

In [ ]:
%%time
for filt in [True, False]:
    model_rnk = reciprocal_rank(model, interactions_valid, k=k, filter_previous=filt)
    model_pre = precision(model, interactions_valid, k=k, filter_previous=filt)
    model_rec = recall(model, interactions_valid, k=k, filter_previous=filt)
    print(f"model reciprocal rank(filter_previous={str(filt)}): {model_rnk:.3f}")
    print(f"model precision(filter_previous={str(filt)}): {model_pre:.3f}")
    print(f"model recall(filter_previous={str(filt)}): {model_rec:.3f}")

## Single function

In [ ]:
rfmlib.topn_recommendations(model, interactions_dct, base_rec)

# Additional Experiments

#### Assess the Diversity of Recommendations

In [ ]:
model_diversity = diversity(model, interactions_valid, k=k).rename({'item_id': 'product_id'}, axis=1)
model_diversity = pd.merge(model_diversity, products_df, on='product_id', how='inner')
model_diversity = model_diversity[['cnt_users', 'pct_users', 'product_id', 'product_name', 'aisle_id', 'department_id']]
model_diversity.head(20)

In [ ]:
coverage = np.mean(model_diversity['cnt_users'] > 0)
round(coverage, 3)

In [ ]:
nonzero = model_diversity[model_diversity.cnt_users > 0]
entropy = -np.sum(nonzero['pct_users'] * np.log2(nonzero['pct_users']))
round(entropy, 2)

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=[16, 4])
N = 50

topN = model_diversity.iloc[:N, :]
axes.bar(topN.index.values + 1, topN.pct_users, width=1, edgecolor='black', alpha=0.75)
axes.set(xlabel='Item Rank', ylabel='Percentage of Users', title='Percentage of Users Recommended by Item Rank')
plt.show()

#### Get Similar Items for a Few Items

In [ ]:
random_user = np.random.choice(valid_users)
print("random user: {}".format(random_user))

In [ ]:
random_user_recs = valid_recs.loc[random_user]
random_user_recs = products_df[products_df.product_id.isin(random_user_recs)].set_index('product_id').loc[random_user_recs]
random_user_recs

#### Look at Similar Products for a Few of the Random User's Recommended Products

In [ ]:
most_similar_items = model.similar_items(48370)
most_similar_items = products_df.set_index('product_id').loc[most_similar_items]
most_similar_items

### Asess Model Performance on Novel Item Recommendations

#### Save an Array of the Most Popular Items in the Training Data

In [ ]:
most_popular = interactions_train.groupby('product_id')['user_id'].count().sort_values(ascending=False)
print(len(most_popular))
most_popular.head(10)

In [ ]:
most_popular = pd.Series(most_popular.index.values)
most_popular[:10]

#### Save the TopK Most Popular Items Not Yet Previously Purchased for Each Customer

In [ ]:
train_user_items = interactions_train.groupby('user_id')['product_id'].apply(set).to_dict()
train_user_items = {key: val for key, val in test_user_items.items() if key in set(valid_users)}
most_popular_new = {user: most_popular[~most_popular.isin(train_user_items[user])][:k] for user in train_user_items.keys()}
len(most_popular_new)

In [ ]:
list(most_popular_new.keys())[:10]

In [ ]:
most_popular_new[100232578]

#### Calculate the Pure-Popularity Baseline Metrics on Previously Unpurchased Items

In [ ]:
base_new_hrt = np.mean([int(len(set(most_popular_new[user]) & set(test_user_items[user])) > 0)                           for user in test_user_items.keys()])
base_new_pre = np.mean([len(set(most_popular_new[user]) & set(test_user_items[user])) / len(set(most_popular_new[user])) for user in test_user_items.keys()])
base_new_rec = np.mean([len(set(most_popular_new[user]) & set(test_user_items[user])) / len(set(test_user_items[user]))  for user in test_user_items.keys()])

In [ ]:
print("number of test users: {}".format(len(test_user_items)))
print("baseline new hit rate: {:.3f}".format(base_new_hrt))
print("baseline new precision: {:.3f}".format(base_new_pre))
print("baseline new recall: {:.3f}".format(base_new_rec))

#### Calculate Model Performance Excluding Training Items

In [ ]:
%%time
model_hrt_new = hit_rate(model, interactions_valid, k=k, filter_previous=True)
model_pre_new = precision(model, interactions_valid, k=k, filter_previous=True)
model_rec_new = recall(model, interactions_valid, k=k, filter_previous=True)

In [ ]:
print("model new hit rate: {}".format(round(model_hrt_new, 3)))
print("model new precision: {}".format(round(model_pre_new, 3)))
print("model new recall: {}".format(round(model_rec_new, 3)))